In [1]:
import pandas as pd
import numpy as np
import datetime

#%matplotlib inline

# Leo csv

In [2]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [3]:
#avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','descripcion','titulo','denominacion_empresa','idpais'],axis=1)

In [4]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [5]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Borro duplicados y agrego edad

In [6]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [7]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

# En un principio los relleno con la media de los postulantes
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad                   0
dtype: int64

In [8]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)
avisos_detalle.isnull().sum()
avisos_detalle.loc[avisos_detalle['nivel_laboral'].isnull(),'nivel_laboral'] = 'Otro'
avisos_detalle.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1000872556,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción
1,9240880,Capital Federal,Full-time,Jefe / Supervisor / Responsable,Comercial
2,1110185164,Gran Buenos Aires,Full-time,Junior,Administración
3,1110513885,Gran Buenos Aires,Full-time,Junior,Distribución
4,1111034024,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial


In [9]:
postulaciones = postulaciones.drop_duplicates(keep='last')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

# Separo gente postulada y no postulada

In [10]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [11]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [12]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [13]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [14]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
#gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [15]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [ ]:
gente_postulada.isnull().sum()

# Arreglo el df de avisos detalle

In [ ]:
avisos_detalle['nombre_area'].unique().shape

In [ ]:
avisos_detalle['nombre_area'].unique()

In [16]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [ ]:
print(avisos_detalle['nombre_area'].value_counts())

In [17]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

In [ ]:
print(avisos_detalle['nombre_area'].value_counts())

# Funciones

In [18]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [19]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

In [20]:
def feature_area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

# Funciones utilidad

In [21]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [22]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

# Tupla postulada y sample no postulado

In [23]:
tupla_postulada = postulaciones.drop('fechapostulacion',axis=1)
tupla_postulada['y'] = 1
tupla_postulada = feature_genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = tupla_postulada.drop(['fechanacimiento'],axis=1)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [ ]:
tupla_postulada.head()

In [24]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],10,1000)
sample['y'] = 0
sample = feature_genero_edad(sample,gente_no_postulada)
sample = sample.drop(['fechanacimiento'],axis=1)
sample = oneHotEncoding(sample,'sexo')

In [ ]:
sample.head()

# Borro lo que no necesito

In [25]:
#del avisos_detalle
del postulantes_genero_y_edad
del postulaciones

# Separo la edad por rango en el sample y el test

In [26]:
sample = sample.append(tupla_postulada.sample(n=100000,random_state=0))

In [27]:
sample = sample.drop('idpostulante',axis=1)

In [28]:
sample['edad'] = pd.cut(sample['edad'], range(0, 100, 5))
sample['edad'] = sample['edad'].astype(str)
dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}
sample['edad'] = sample['edad'].map(dic)

In [29]:
sample['edad'] = sample['edad'].fillna(2)

In [30]:
sample = feature_area_de_trabajo(sample)

In [31]:
sample = oneHotEncoding(sample,'nombre_area')

In [32]:
sample = oneHotEncoding(sample,'nivel_laboral')

In [33]:
sample = oneHotEncoding(sample,'nombre_zona')

In [34]:
sample = oneHotEncoding(sample,'tipo_de_trabajo')

In [35]:
sample = sample.drop(['idaviso'],axis=1)

In [ ]:
sample.head()

# Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=500,random_state=0, n_jobs=-1)
scores = cross_val_score(clf, sample.drop(columns='y'),sample['y'], cv=5)

In [ ]:
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
print(clf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(sample.drop(columns='y'), sample['y'])

In [ ]:
rf_random.best_params_

# Grid search y randomized search

In [36]:
print(__doc__)

import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier


An interactive shell.

This file is designed to be launched as a PYTHONSTARTUP script.



In [37]:
X, y = sample.drop(columns='y'), sample['y']

In [38]:
# build a classifier
clf = RandomForestClassifier(n_estimators=20)

In [39]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [40]:
# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

In [42]:
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 287.41 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.674 (std: 0.003)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 6, 'min_samples_leaf': 9, 'min_samples_split': 10}

Model with rank: 2
Mean validation score: 0.673 (std: 0.004)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 6, 'min_samples_split': 4}

Model with rank: 3
Mean validation score: 0.672 (std: 0.004)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 8, 'min_samples_leaf': 2, 'min_samples_split': 2}



In [41]:
# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

In [43]:
# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

GridSearchCV took 2820.43 seconds for 216 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.674 (std: 0.004)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 10}

Model with rank: 2
Mean validation score: 0.674 (std: 0.004)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 2}

Model with rank: 3
Mean validation score: 0.674 (std: 0.004)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 3}

